# Object Detection API Demo

<table align="left"><td>
  <a target="_blank"  href="https://github.com/tensorflow/models/blob/master/research/object_detection/object_detection_tutorial.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab
  </a>
</td><td>
  <a target="_blank"  href="https://colab.sandbox.google.com/github/tensorflow/models/blob/master/research/object_detection/object_detection_tutorial.ipynb">
    <img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
</td></table>

Welcome to the [Object Detection API](https://github.com/tensorflow/models/tree/master/research/object_detection). This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image.

> **Important**: This tutorial is to help you through the first step towards using [Object Detection API](https://github.com/tensorflow/models/tree/master/research/object_detection) to build models. If you just just need an off the shelf model that does the job, see the [TFHub object detection example](https://colab.sandbox.google.com/github/tensorflow/hub/blob/master/examples/colab/object_detection.ipynb).

# Setup

Important: If you're running on a local machine, be sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md). This notebook includes only what's necessary to run in Colab.

### Install

Get `tensorflow/models` or `cd` to parent directory of the repository.

In [1]:
thresh_hold_val = float(input("Enter value 0-1: "))

Enter value 0-1: 0.3


In [2]:
import os
import pathlib


if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Compile protobufs and install the object_detection package

### Imports

In [3]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display
from collections import Counter

Import the object detection module.

In [4]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

Patches:

In [5]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing the path.

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

## Loader

In [6]:
def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)

  model_dir = pathlib.Path(model_dir)/"saved_model"

  model = tf.saved_model.load(str(model_dir))
  model = model.signatures['serving_default']

  return model

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [7]:
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = r'F:\models-master\research\object_detection\data\mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

For the sake of simplicity we will test on 2 images:

# Detection

Load an object detection model:

In [8]:
model_name = 'ssd_resnet50_v1_fpn_shared_box_predictor_640x640_coco14_sync_2018_07_03'
detection_model = load_model(model_name)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Check the model's input signature, it expects a batch of 3-color images of type uint8: 

Run it on each test image and show the results:

## realtime object detection

In [9]:
import cv2
from lxml import etree
import xml.etree.cElementTree as ET


In [10]:
#r'C:\Users\LENOVO\Desktop\dolly vidios\car1.mp4'

In [15]:
def playvideo(videofile,location,frame_number):
    cap = cv2.VideoCapture(videofile)
    count= 0
    zero_array = [0]*2
    items = ['person','car']
    sourceDirectory = r'C:\Users\LENOVO\Desktop\dolly vidios\yolo\labels'
    sourceDirectory2 = r'C:\Users\LENOVO\Desktop\dolly vidios\yolo\yololabels'
    while cap.isOpened():
        ret, image_np = cap.read()
        ret1,frame = cap.read()
          # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
        if ret:
            image = np.asarray(image_np)
          # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
            input_tensor = tf.convert_to_tensor(image)
          # The model expects a batch of images, so add an axis with `tf.newaxis`.
            input_tensor = input_tensor[tf.newaxis,...]

          # Run inference
            output_dict = detection_model(input_tensor)

          # All outputs are batches tensors.
          # Convert to numpy arrays, and take index [0] to remove the batch dimension.
          # We're only interested in the first num_detections.
            num_detections = int(output_dict.pop('num_detections'))
            output_dict = {key:value[0, :num_detections].numpy() 
                         for key,value in output_dict.items()}
            output_dict['num_detections'] = num_detections

          # detection_classes should be ints.
            output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
              # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
              image_np,
              output_dict['detection_boxes'],
              output_dict['detection_classes'],
              output_dict['detection_scores'],
              category_index,
              instance_masks=output_dict.get('detection_masks_reframed', None),
              use_normalized_coordinates=True,
              min_score_thresh=thresh_hold_val,
              line_thickness=8)



            image,boxes,classname,classes = vis_util.visualize_boxes_and_labels_on_image_array(image_np,
                                                                           output_dict['detection_boxes'],
                                                                           output_dict['detection_classes'],
                                                                           output_dict['detection_scores'],
                                                                           category_index,
                                                                           instance_masks=output_dict.get('detection_masks_reframed', None),
                                                                           use_normalized_coordinates=True,
                                                                           min_score_thresh=thresh_hold_val,
                                                                           line_thickness=8)

            img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)



            im = Image.fromarray(img)
            width,height = im.size

            
            annotation = ET.Element('annotation')
            ET.SubElement(annotation, 'folder').text = 'kitty'
            ET.SubElement(annotation, 'filename').text = str(count)+'.jpg'
            ET.SubElement(annotation, 'segmented').text = '0'
            size = ET.SubElement(annotation, 'size')
            ET.SubElement(size, 'width').text = str(width)
            ET.SubElement(size, 'height').text = str(height)
            ET.SubElement(size, 'depth').text = '3'
            
            
            
            lines = []
            yolotext = []
            for i in boxes:
              left =  int(i[1]*width)
              bottom = int(i[2]*height)
              right =int(i[3]*width)
              top = int(i[0]*height)
              val = (left, top, right, bottom)
              im1 = im.crop((left, top, right, bottom))

              index = boxes.index(i)
              object_name = classname[index]
              classid = classes[index]
              if classid == 3:
                  classid = 0
              path = location+'/'+object_name    #r'C:\Users\LENOVO\Desktop\dolly

              objindex= items.index(object_name)
              zero_array[objindex] +=1
              a = zero_array[objindex]  
              
              dw = 1./width
              dh = 1./height
              
              x = (left+right)/2.0
              y = (top+bottom)/2.0
              w = right - left
              h = bottom - top
              
              x = x*dw
              y = y*dh
              w = w*dw
              h = h*dh
              
              yolotext.append(str(classid)+' '+str(round(x,3))+' '+str(round(y,3))+' '+str(round(w,3))+' '+str(round(h,3)))
              lines.append(object_name + ' ' + '0.00 0 0.00 ' +str(left) + ' ' + str(top) + ' ' + str(right) + ' ' + str(bottom) + ' 0.00 0.00 0.00 0.00 0.00 0.00 0.00')
                           
              if not os.path.exists(path):
                 os.makedirs(path)
              im1.save(path+'/'+object_name+str(a)+'.jpg')  
            
              ob = ET.SubElement(annotation, 'object')
              ET.SubElement(ob, 'name').text = object_name
              ET.SubElement(ob, 'pose').text = 'Unspecified'
              ET.SubElement(ob, 'truncated').text = '0'
              ET.SubElement(ob, 'difficult').text = '0'
              bbox = ET.SubElement(ob, 'bndbox')
              ET.SubElement(bbox, 'xmin').text = str(left)
              ET.SubElement(bbox, 'ymin').text = str(top)
              ET.SubElement(bbox, 'xmax').text = str(right)
              ET.SubElement(bbox, 'ymax').text = str(bottom)

            xml_str = ET.tostring(annotation)
            root = etree.fromstring(xml_str)
            xml_str = etree.tostring(root, pretty_print=True)
            xml_labels = str(count) + '.xml'
            save_path = os.path.join(sourceDirectory, xml_labels)
            with open(save_path, 'wb') as temp_xml:
                temp_xml.write(xml_str)
              
            
            
            
            
            yolo_label = str(count)+ '.txt'
            with open(os.path.join(sourceDirectory2, yolo_label), "w") as text12_file:
                for item in yolotext:
                    text12_file.write("%s\n" % item)              
            

            kitti_labels = str(count) + '.txt'
            with open(os.path.join(sourceDirectory, kitti_labels), "w") as text_file:
                for item in lines:
                    text_file.write("%s\n" % item)
            im.save(sourceDirectory+'/'+str(count)+'.jpg')
            count += frame_number         #30 # i.e. at 30 fps, this advances one second
            cap.set(1, count)






            cv2.imshow('object detection',image_np)
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
        else:
            cap.release()
            break
    #cap.release()
    cv2.destroyAllWindows()

In [16]:
videofile = input('enter the videofile input path')
location = input('enter the path location to save images')
frame_number = int(input('enter the frame number'))
playvideo(videofile,location,frame_number)

enter the videofile input pathC:\Users\LENOVO\Desktop\dolly vidios\car1.mp4
enter the path location to save imagesC:\Users\LENOVO\Desktop\dolly vidios\yolo
enter the frame number30


In [ ]:
#C:\Users\LENOVO\Desktop\dolly vidios\yolo
